# Windy Gridworld

Imagine a standard gridworld, with start and goal states, but with one difference: there is a crosswind running upward through the middle of the grid. The actions are the standard four -`up`, `down`, `right`, `left`- but in the middle region the resultant next states are shifted upward by "wind", the strength of which varies from column to column.

This is an undiscounted episodic task, with **constant rewards of -1** until the goal is reached.

## Windy Gridworld Static Wind

Let's say the first 3 columns don't have wind, the next 3 columns have wind by 1, then the next 2 columns have wind by 2, then 1 and finally 0. So, we have a griworld of 10 columns.

In [1]:
from TD_methods import td_sarsa_control
import numpy as np

In [2]:
# Let's create just the gridworld
gridworld = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 2, 2, 1, 0, 0] # This row represents the wind in each column
])

In [3]:
n, m = gridworld.shape

Say that a state is represented by a tuple, which has the position in the first index, and the wind in the second.

In [4]:
non_terminal_states=[]
terminal_states=[]
initial_states=[]
for i in range(n-1):
    for j in range(m):
        if gridworld[i, j] != 0:
            if gridworld[i, j] == 1:
                non_terminal_states.append(((i, j), gridworld[n-1, j]))
            elif gridworld[i, j] == 2:
                initial_states.append(((i, j), gridworld[n-1, j]))
                non_terminal_states.append(((i, j), gridworld[n-1, j]))
            else:
                terminal_states.append(((i, j), gridworld[n-1, j]))

In [5]:
actions={}
states = non_terminal_states + terminal_states
for state in states:
    actions[state] = [action for action in ['up', 'down', 'right', 'left']]

In [6]:
def next_step_static_wind(state, action):
    """
    Takes as input a state and an action, and returns a (state, reward) tuple that follows the state-action pair.
    """
    def invalid_position(position: tuple):
        if gridworld[position] == 0:
            return True
    i, j = state[0]
    wind = state[1]

    if action == 'up':
        new_position = (i + 1 + wind, j)
    elif action == 'down':
        new_position = (i - 1 + wind, j)
    elif action == 'right':
        new_position = (i + wind, j + 1)
    else:
        new_position = (i + wind, j - 1)
    
    if invalid_position(new_position):
        return (state, -1)
    # The next state depends on thw wind, and wind depends on the 'j'
    new_wind = gridworld[n-1, new_position[1]]
    return (((new_position), new_wind), -1)

In [8]:
Q, policy = td_sarsa_control(non_terminal_states=non_terminal_states,
                 terminal_states=terminal_states,
                 initial_states=initial_states,
                 actions=actions,
                 next_step_fn=next_step_static_wind,
                 gamma=1.0,
                 alpha=0.5,
                 epsilon=0.1,
                 num_episodes=100000,
                 initial_Q=None)

0


KeyboardInterrupt: 

In [23]:
initial_state = initial_states[0]
policy[initial_state]

'right'

In [25]:
initial_state

((4, 1), np.int64(0))

In [24]:
next_step_static_wind(initial_state, policy[initial_state])

(((5, np.int64(1)), np.int64(0)), -1)